In [1]:
import igraph as ig
import numpy as np
import tables as tb
from faiss import IndexFlatL2, IndexIVFFlat, omp_set_num_threads
from numpy.typing import NDArray

In [2]:
%load_ext watermark
%watermark -vp numpy,igraph,tables,faiss

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.13.2

numpy : 1.23.5
igraph: 0.11.3
tables: 3.8.0
faiss : 1.8.0



In [3]:
FloatArray = NDArray[np.float32]
LongArray = NDArray[np.int64]
THREADS = 128

omp_set_num_threads(THREADS)

def build_index(data: FloatArray, n_cells: int) -> IndexIVFFlat:
    quantizer = IndexFlatL2(data.shape[-1])
    index = IndexIVFFlat(quantizer, data.shape[-1], n_cells)
    index.train(data)  # type: ignore
    index.add(data)  # type: ignore
    return index


def to_sparse(idx: LongArray, sim: FloatArray) -> ig.Graph:
    _adj = []
    _total_mask = []
    for i, (row_idx, row_sim) in enumerate(zip(idx, sim)):
        mask = (row_idx >= 0) & (row_sim > 0)  # ignore 0 similarity edges
        row_idx = row_idx[mask]

        dsize = row_idx.shape[0]
        d = np.vstack((np.repeat(i, dsize), row_idx))

        _adj.append(d)
        _total_mask.append(mask)

    adj = np.hstack(_adj)
    total_mask = np.hstack(_total_mask)

    edge_attr = sim.reshape(-1)[total_mask]

    n_nodes = idx.shape[0]
    g = ig.Graph(n_nodes, adj.T)
    g.es["weight"] = edge_attr
    return g


def knn(data: FloatArray, index: IndexIVFFlat, k: int) -> ig.Graph:
    dist: FloatArray
    idx: LongArray
    # faiss includes self hits so need k + 1 neighbors
    dist, idx = index.search(data, k=k + 1)  # type: ignore
    scale = 1 / np.sqrt(data.shape[-1])
    # upcast to prevent overflow
    rbf = np.exp(-(np.square(dist.astype(np.float64))) * scale)

    return to_sparse(idx, rbf)


def leiden_clustering(graph: ig.Graph, resolution: float) -> LongArray:
    output = graph.community_leiden(weights="weight", resolution=resolution)
    labels = output.membership
    return np.array(labels)

In [4]:
# just need any (n genomes, d features) genome embedding matrix
with tb.open_file("../../../genome_repr/IMGVRv4_test_set_esm2_t30_150M_accum50_results.h5") as fp:
    genome_embed = fp.root.data[:]

# n genomes x d features
genome_embed.shape

(151255, 1280)

In [5]:
n_cells = 3875
index = build_index(genome_embed, n_cells)
graph = knn(genome_embed, index, k=15)

In [6]:
# (n genomes, ) dimensional array that just has a cluster label for each genome
cluster_labels = leiden_clustering(graph, resolution=1.0)
cluster_labels.shape, cluster_labels

((151255,), array([    0,     1,     2, ..., 41671, 20422, 15804]))

In [7]:
# cluster sizes
np.bincount(cluster_labels)

array([3, 7, 2, ..., 1, 1, 1])